# boto3でS3のcsvをさわる && pyathenaでS3上のcsvにSQL発行
---

- pandas.DataFrame <--> s3
    - s3 -->> pd.DataFrame
    - pd.DataFrame -->> s3に新しくcsvファイルをput
    - pd.DataFrame -->> s3のcsvを更新

AWSでユーザかロールか作ってアクセスキーとかを控えておきます。

事前にS3にcsvファイルを置いときました。

- pythenaでS3のcsvにSQL

---

## import && awsのaccess keyとかを環境変数へ && 定数

今回はdockerコンテナ上でpythonしててマウントしたりコンテナ内にaws cli入れてcongfigureがめんどくさいので、アクセスキーとかここで環境変数に書いています。

ローカル等でやるときはaws cliで`aws configure`してしまうのがよさそう。

In [1]:
import io
import boto3
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
## constant value

bucket = 'testbktforme'
key_dir = 'dataset'
key = 'dataset/wine.csv'
newkey = 'dataset/testdata.csv'

aws_access_key = os.environ["AWS_ACCESS_KEY_ID"]
aws_secret_key = os.environ["AWS_SECRET_ACCESS_KEY"]
aws_region = os.environ['AWS_REGION']
s3_staging_dir = os.environ['S3_STAGING_DIR']

---

## read_s3, to_s3を関数定義

- read_s3()

    boto3でcsvのbody持ってきてread_csvでDataFrame化してる

- to_s3()

    boto3でS3にputしてる


以下url参照した

https://ksomemo.github.io/contents/qiita/pandas%20%E3%81%A7%20read_s3%20%E3%81%A8%20to_s3.html

In [3]:
def read_s3(bucket, key, **kwargs):
    s3_cleint = boto3.client('s3')
    obj = s3_cleint.get_object(Bucket=bucket, Key=key)
    return pd.read_csv(obj['Body'], **kwargs)

def to_s3(df, bucket, key, **kwargs):
    with io.StringIO() as buf:
        df.to_csv(buf, **kwargs)
        s3_client = boto3.client('s3')
        s3_client.put_object(
            Bucket=bucket, Key=key,
            Body=io.BytesIO(buf.getvalue().encode()))

---

## s3 -->> pd.DataFrame

事前にs3にwine.csvおいてある

In [14]:
df = read_s3(bucket, key)
df.columns = ['id', 'alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash',
       'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline']
df

,id,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [17]:
to_s3(df, 'testbktforme', 'test/test.csv', index=False)

---

## pd.DataFrame -->> s3に新しくcsvファイルをput

In [6]:
## 適当にdfもってくる
## ここではsklearnのサンプルデータを使ってみる。
from sklearn.datasets import load_iris
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)

## to_s3
to_s3(df=df, bucket=bucket, key=newkey)

In [7]:
## もっかい持ってきて確認
df = read_s3(bucket, newkey)
df

,Unnamed: 0,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,5.1,3.5,1.4,0.2
1,1,4.9,3.0,1.4,0.2
2,2,4.7,3.2,1.3,0.2
3,3,4.6,3.1,1.5,0.2
4,4,5.0,3.6,1.4,0.2
...,...,...,...,...,...
145,145,6.7,3.0,5.2,2.3
146,146,6.3,2.5,5.0,1.9
147,147,6.5,3.0,5.2,2.0
148,148,6.2,3.4,5.4,2.3


---

## pd.DataFrame -->> s3のcsvを更新

In [8]:
## 上のdfのdescribe()をputすることにする
df = df.describe()

## to_s3
to_s3(df, bucket, newkey)

In [9]:
## もっかい持ってきて確認
df = read_s3(bucket, newkey)
df

,Unnamed: 0,Unnamed: 0.1,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,count,150.000000,150.000000,150.000000,150.000000,150.000000
1,mean,74.500000,5.843333,3.057333,3.758000,1.199333
2,std,43.445368,0.828066,0.435866,1.765298,0.762238
3,min,0.000000,4.300000,2.000000,1.000000,0.100000
4,25%,37.250000,5.100000,2.800000,1.600000,0.300000
5,50%,74.500000,5.800000,3.000000,4.350000,1.300000
6,75%,111.750000,6.400000,3.300000,5.100000,1.800000
7,max,149.000000,7.900000,4.400000,6.900000,2.500000


---

## pyathena使ってみる

pyathenaからaws athenaにクエリ。athenaに準備が必要、

なんかうまくいかん。athena側の設定だと思う

該当S3にGlueでクローラを設定したりすると自動で色々できそう

https://aws.amazon.com/jp/blogs/news/run-sql-queries-from-your-sagemaker-notebooks-using-amazon-athena/

In [10]:
# ! pip install pyathena

In [18]:
from pyathena import connect
import pandas as pd

# AWS接続情報
AWS_ACCESS_KEY = aws_access_key
AWS_SECRET_ACCESS_KEY = aws_secret_key
AWS_REGION_NAME = aws_region
AWS_S3_STAGING_DIR = s3_staging_dir

cn = connect(aws_access_key_id=AWS_ACCESS_KEY,
             aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
             s3_staging_dir=AWS_S3_STAGING_DIR,
             region_name=AWS_REGION_NAME)

# 実行SQL
sql = "SELECT * FROM test.test"

# DataFrameとして取得
df = pd.read_sql(sql, cn)
df

,id,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
